In [1]:
from transformers import pipeline
import torchaudio
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch
from huggingface_hub import login
from dotenv import load_dotenv

In [ ]:
load_dotenv()

HF_TOKEN = os.getenv('HF_TOKEN')
login(HF_TOKEN, add_to_git_credential=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
asr = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device, return_timestamps=True)

In [ ]:
audio_path = "./denver_extract.mp3 - Google Drive.mp3"

In [ ]:
waveform, sample_rate = torchaudio.load(audio_path)

if sample_rate != 16000:
    print(f"Resampling from {sample_rate} Hz to 16000 Hz...")
    resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
    waveform = resampler(waveform)
    audio_path = "resampled_audio.wav"
    torchaudio.save(audio_path, waveform, 16000)

In [ ]:
print("Transcribing...")
result = asr(audio_path)

In [ ]:
print("\n Transcribed Text:\n")
print(result["text"])

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)